<a href="https://colab.research.google.com/github/mystlee/2024_CSU_AI/blob/main/crab_age_prediction/2024_AI_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

kaggle에서 baseline 코드가 안보이는 관계로 부득이하게 colab에서 실행   

files.upload()까지 코드가 진행되면, kaggle login창이 나타나고 로그인 진행   
이전에 kaggle 사이트에서 프로필 사진 클릭 후 -> settings 진입 -> create new token
여기서 생성된 kaggle.json을 업로드
그러면 private 데이터라고 하더라도 현재 colab으로 들고올 수 있음


In [18]:
!pip install kaggle

from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c 2024-csu-iceai-crab-age-regression-competition

!mkdir -p /kaggle/input/2024-csu-iceai-crab-age-regression-competition
!mkdir -p /kaggle/output/2024-csu-iceai-crab-age-regression-competition
!unzip 2024-csu-iceai-crab-age-regression-competition.zip -d /kaggle/input/2024-csu-iceai-crab-age-regression-competition

2024-csu-iceai-crab-age-regression-competition.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  2024-csu-iceai-crab-age-regression-competition.zip
  inflating: /kaggle/input/2024-csu-iceai-crab-age-regression-competition/sample_submission.csv  
  inflating: /kaggle/input/2024-csu-iceai-crab-age-regression-competition/test_v2.csv  
  inflating: /kaggle/input/2024-csu-iceai-crab-age-regression-competition/train.csv  


## 환경 설정   
라이브러리 설치   
pandas, scikt-learn

In [13]:
!pip install pandas scikit-learn

## 0. 필요한 라이브러리 호출

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [15]:
!ls /kaggle/input/2024-csu-iceai-crab-age-regression-competition

sample_submission.csv  test_v2.csv  train.csv


## 1-1. 데이터셋 로드

In [19]:
data = pd.read_csv('/kaggle/input/2024-csu-iceai-crab-age-regression-competition/train.csv')
test_data = pd.read_csv('/kaggle/input/2024-csu-iceai-crab-age-regression-competition/test_v2.csv')

print(data.head())

   id Sex  Length  Diameter  Height     Weight  Shucked Weight  \
0   0   M  1.6375     1.300  0.4750  38.711242       17.690088   
1   1   M  1.5375     1.175  0.3750  33.409886       11.495722   
2   2   F  1.7125     1.375  0.4750  43.629880       16.726205   
3   3   F  1.6375     1.275  0.4375  38.980562       16.683681   
4   4   F  1.7125     1.375  0.4250  51.511041       22.254358   

   Viscera Weight  Shell Weight   Age  
0        7.413394     11.339800  10.0  
1        8.022908      8.930093  17.0  
2       11.424848     14.033003  13.0  
3        6.676307     11.580771  11.0  
4       11.439023     12.332033  10.0  


## 1-2. 데이터셋 전처리

In [20]:
le = LabelEncoder()
data['Sex'] = le.fit_transform(data['Sex'])
test_data['Sex'] = le.fit_transform(test_data['Sex'])

X = data[['Sex', 'Length', 'Diameter', 'Height', 'Weight',
          'Shucked Weight', 'Viscera Weight', 'Shell Weight']]
y = data['Age']

X_test = test_data[['Sex', 'Length', 'Diameter', 'Height', 'Weight',
          'Shucked Weight', 'Viscera Weight', 'Shell Weight']]
y_test = ''

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state = 42)

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

## 3.a. Decision tree를 이용한 regression model

In [21]:
dt_regressor = DecisionTreeRegressor(random_state = 42)
dt_regressor.fit(X_train, y_train)
y_pred_dt = dt_regressor.predict(X_valid)

## 3.b. support vector machine를 이용한 regression model

In [22]:
svm_regressor = SVR()
svm_regressor.fit(X_train_scaled, y_train)
y_pred_svm = svm_regressor.predict(X_valid_scaled)

## 4. 검증 데이터를 이용한 평가

In [23]:
mae_dt = mean_absolute_error(y_valid, y_pred_dt)
print('Public dataset score')
print("Decision Tree Regressor")
print(f"Mean Absolute Error: {mae_dt:.4f}")

mae_svm = mean_absolute_error(y_valid, y_pred_svm)
print("Support Vector Machine Regressor")
print(f"Mean Absolute Error: {mae_svm:.4f}")

Public dataset score
Decision Tree Regressor
Mean Absolute Error: 1.8652
Support Vector Machine Regressor
Mean Absolute Error: 1.3040


## 5. 제출 파일 생성

In [25]:
y_sub_dt = dt_regressor.predict(X_test)
results = pd.DataFrame({
    'id': range(len(y_sub_dt)),
    'Age': np.int_(y_sub_dt),
})
results.to_csv('dt_submission.csv', index = False)

y_sub_svm = svm_regressor.predict(X_test_scaled)
results = pd.DataFrame({
    'id': range(len(y_sub_svm)),
    'Age': np.int_(y_sub_svm),
})
results.to_csv('svm_submission.csv', index = False)

files.download('dt_submission.csv')
files.download('svm_submission.csv')